# Quantum Search Algorithms Project using Qiskit

## 1. Setup Environment and Import Required Libraries

In [ ]:
# Install Qiskit if not already installed
# !pip install qiskit qiskit[visualization]

from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit.algorithms import Grover, AmplificationProblem

## 2. Classical Baselines

In [ ]:
def classical_linear_search(lst, target):
    for i, item in enumerate(lst):
        if item == target:
            return i
    return -1

# Test classical linear search
items = ['a', 'b', 'c', 'd', 'target', 'e']
print("Classical index found:", classical_linear_search(items, 'target'))

In [ ]:
def is_balanced_or_constant(classical_f, n):
    results = [classical_f(x) for x in range(2**n)]
    if all(r == results[0] for r in results):
        return "Constant"
    elif results.count(0) == results.count(1):
        return "Balanced"
    else:
        return "Neither"


## 3. Deutsch-Jozsa Algorithm

In [ ]:
def deutsch_jozsa(oracle, n):
    qc = QuantumCircuit(n + 1, n)
    qc.x(n)
    qc.h(range(n + 1))
    qc.compose(oracle, inplace=True)
    qc.h(range(n))
    qc.measure(range(n), range(n))
    return qc

In [ ]:
def balanced_oracle(n):
    qc = QuantumCircuit(n + 1)
    for i in range(n):
        qc.cx(i, n)
    return qc

In [ ]:
def constant_oracle(n):
    qc = QuantumCircuit(n + 1)
    return qc

In [ ]:
oracle = balanced_oracle(3)  # You can try constant_oracle(3) too
qc_dj = deutsch_jozsa(oracle, 3)
qc_dj.draw('mpl')

In [ ]:
sim = Aer.get_backend('qasm_simulator')
result_dj = execute(qc_dj, backend=sim, shots=1024).result()
counts_dj = result_dj.get_counts()
plot_histogram(counts_dj)
plt.show()

## 4. Grover's Algorithm

In [ ]:
def create_oracle(n, marked_element):
    oracle = QuantumCircuit(n)
    binary = format(marked_element, f'0{n}b')
    for i, bit in enumerate(binary):
        if bit == '0':
            oracle.x(i)
    oracle.h(n - 1)
    if n == 3:
        oracle.ccx(0, 1, 2)
    oracle.h(n - 1)
    for i, bit in enumerate(binary):
        if bit == '0':
            oracle.x(i)
    oracle_gate = oracle.to_gate()
    oracle_gate.name = "Oracle"
    return oracle_gate

In [ ]:
def grover_search(marked_element, n=3):
    oracle_gate = create_oracle(n, marked_element)
    problem = AmplificationProblem(oracle=oracle_gate, is_good_state=lambda x: x == format(marked_element, f'0{n}b'))
    grover = Grover()
    result = grover.amplify(problem)
    print("Grover result:", result.assignment)

grover_search(5)

## 5. Analysis and Report
- Compare runtime and complexity.
- Discuss quantum speedup, decoherence, and circuit depth.
- Document your results for presentation and reporting.